In [1]:
import re
import math
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score ,precision_score , recall_score , confusion_matrix , f1_score
from sklearn.model_selection import GridSearchCV
from ast import literal_eval
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
pca = PCA(n_components=40)
from sklearn import datasets, metrics, model_selection
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#data_train = pd.read_csv('./data/twitter_data_before_analysis.csv', error_bad_lines=False)
data_train = pd.read_csv('./data/twitter_data_before_analysis1.csv', error_bad_lines=False)
data_train = data_train.append(pd.read_csv('./data/ko_news_data_before_analysis.csv', error_bad_lines=False))
data_train = data_train.append(pd.read_csv('./data/en_news_data_before_analysis.csv', error_bad_lines=False))

In [3]:
data_train['period'] = pd.to_numeric(data_train['period'], errors='coerce')
data_train['category'] = pd.to_numeric(data_train['category'], errors='coerce')
data_train['label'] = pd.to_numeric(data_train['label'], errors='coerce')
data_train['Silhouette'] = pd.to_numeric(data_train['Silhouette'], errors='coerce')

In [4]:
x_data_df = data_train[data_train.columns.difference(['label'])]

In [5]:
#x_data_df = data_train[['period','raw_data','closer#','category','label','Silhouette']]
y_data_df = data_train['label']

In [6]:
x_data_df = x_data_df.drop('Unnamed: 0', axis=1)
x_data_df = x_data_df.drop('result', axis=1)
x_data_df = x_data_df.drop('Silhouette', axis=1)

In [7]:
#x_data_df['raw_data'] = x_data_df['raw_data'].apply(lambda x: x[1:len(x)-1].split(','))
print(len(x_data_df)) 

187533


In [8]:
X_train,X_test, y_train, y_test = train_test_split(x_data_df,y_data_df,test_size=0.2,random_state=11)

In [9]:
rf_clf = RandomForestClassifier(random_state = 11 ,n_jobs=-1) # random forest

In [10]:
rf_clf.fit(X_train, y_train)

y_score = rf_clf.predict_proba(X_test)



In [11]:
rf_pred = rf_clf.predict(X_test)  # random forest predict

In [12]:
print('Random Forest Accuracy :  ',accuracy_score(y_test,rf_pred))
print('Random Forest Precision :  ',precision_score(y_test,rf_pred,pos_label='positive',average='weighted'))
print('Random Forest Recall :  ',recall_score(y_test,rf_pred,pos_label='positive',average='weighted'))
print('Random Forest f1 :  ',f1_score(y_test,rf_pred,pos_label='positive',average='weighted'))

Random Forest Accuracy :   0.9418241928173408
Random Forest Precision :   0.941518009537833
Random Forest Recall :   0.9418241928173408
Random Forest f1 :   0.9398852435282192


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [13]:
y_score

array([[0.64, 0.  , 0.  , ..., 0.  , 0.  , 0.17],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.97, 0.  , 0.  , ..., 0.  , 0.01, 0.  ],
       ...,
       [0.  , 0.88, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [14]:
dt_clf = DecisionTreeClassifier(random_state = 11)
dt_clf.fit(X_train,y_train)
dt_pred = dt_clf.predict(X_test)

In [15]:
print('Decision Tree Accuracy :  ',accuracy_score(y_test,dt_pred))
print('Decision Tree Precision :  ',precision_score(y_test,dt_pred,pos_label='positive',average='weighted'))
print('Decision Tree Recall :  ',recall_score(y_test,dt_pred,pos_label='positive',average='weighted'))
print('Decision Tree f1 :  ',f1_score(y_test,dt_pred,pos_label='positive',average='weighted'))

Decision Tree Accuracy :   0.9217746020742794
Decision Tree Precision :   0.9215637369775317
Decision Tree Recall :   0.9217746020742794
Decision Tree f1 :   0.9216394228633673


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


In [16]:
#gb_clf = GradientBoostingClassifier(random_state = 11) # random forest
lgbm_clf =  LGBMClassifier(random_state = 11)

In [17]:
#gb_clf.fit(X_train,y_train)
#gb_pred = gb_clf.predict(X_test)

lgbm_clf.fit(X_train,y_train)
lgbm_pred = lgbm_clf.predict(X_test)

In [18]:
#print('Gredient Boosting Accuracy :  ',accuracy_score(y_test,gb_pred))
#print('Gredient Boosting Precision :  ',precision_score(y_test,gb_pred,pos_label='positive',average='weighted'))
#print('Gredient Boosting Recall :  ',recall_score(y_test,gb_pred,pos_label='positive',average='weighted'))
#print('Gredient Boosting f1 :  ',f1_score(y_test,gb_pred,pos_label='positive',average='weighted'))
print('Light GBM Accuracy :  ',accuracy_score(y_test,lgbm_pred))
print('Light GBM Precision :  ',precision_score(y_test,lgbm_pred,pos_label='positive',average='weighted'))
print('Light GBM Recall :  ',recall_score(y_test,lgbm_pred,pos_label='positive',average='weighted'))
print('Light GBM f1 :  ',f1_score(y_test,lgbm_pred,pos_label='positive',average='weighted'))

Light GBM Accuracy :   0.9512357693230596
Light GBM Precision :   0.9512110500657861
Light GBM Recall :   0.9512357693230596
Light GBM f1 :   0.9508475222230279


In [19]:
lr_clf = LogisticRegression(random_state = 11)

In [20]:
lr_clf.fit(X_train,y_train)
lr_pred = lr_clf.predict(X_test)

In [21]:
print('Logistic Regression Accuracy :  ',accuracy_score(y_test,lr_pred))
print('Logistic Regression  Precision :  ',precision_score(y_test,lr_pred,pos_label='positive',average='weighted'))
print('Logistic Regression  Recall :  ',recall_score(y_test,lr_pred,pos_label='positive',average='weighted'))
print('Logistic Regression  f1 :  ',f1_score(y_test,lr_pred,pos_label='positive',average='weighted'))

Logistic Regression Accuracy :   0.34089636601167783
Logistic Regression  Precision :   0.1162103323599678
Logistic Regression  Recall :   0.34089636601167783
Logistic Regression  f1 :   0.17333231009585082


/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1270: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/Users/hongminpyo/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [28]:
model = Sequential()
model.add(Dense(120, input_dim=103, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(60, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(13, activation='softmax'))

In [29]:
# compile the keras model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
from keras.utils import to_categorical

y_train = y_train.apply(lambda x : x-1)

In [31]:
y_cate_train = to_categorical(y_train , num_classes=13)
#y_cate_train = y_train

In [32]:
model.fit(X_train,y_cate_train, epochs=30, batch_size=500)

Epoch 1/30
150026/150026 [==============================] - 1s 8us/step - loss: 268.5312 - accuracy: 0.1678
Epoch 2/30
150026/150026 [==============================] - 1s 6us/step - loss: 2.3731 - accuracy: 0.3010
Epoch 3/30
150026/150026 [==============================] - 1s 6us/step - loss: 2.2149 - accuracy: 0.3145
Epoch 4/30
150026/150026 [==============================] - 1s 6us/step - loss: 2.0932 - accuracy: 0.3370
Epoch 5/30
150026/150026 [==============================] - 1s 6us/step - loss: 2.0077 - accuracy: 0.3370
Epoch 6/30
150026/150026 [==============================] - 1s 6us/step - loss: 1.9524 - accuracy: 0.3370
Epoch 7/30
150026/150026 [==============================] - 1s 6us/step - loss: 1.9186 - accuracy: 0.3370
Epoch 8/30
150026/150026 [==============================] - 1s 6us/step - loss: 1.8987 - accuracy: 0.3370
Epoch 9/30
150026/150026 [==============================] - 1s 6us/step - loss: 1.8871 - accuracy: 0.3370
Epoch 10/30
150026/150026 [=================

In [33]:
y_test = y_test.apply(lambda x : x-1)

In [34]:
y_cate_test = to_categorical(y_test , num_classes=13)

In [35]:
score = model.evaluate(X_test, y_cate_test, batch_size=500)

37507/37507 [==============================] - 0s 3us/step


In [36]:
score

[2.612895203268569, 0.2952515482902527]

In [37]:
yhat_probs = model.predict(X_test ,batch_size=500 )

yhat_classes = model.predict_classes(X_test ,batch_size=500 )

# reduce to 1d array
#yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 0]

In [38]:
yhat_classes

array([4, 4, 4, ..., 4, 4, 4])

In [39]:
y_cate_test

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes )
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes , average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes , average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes,average='micro')
print('F1 score: %f' % f1)

Accuracy: 0.295252
Precision: 0.295252
Recall: 0.295252
F1 score: 0.295252
